In [ ]:
#| default_exp special
from nbdev import *
from nbdev.showdoc import *
import networkx as nx

# Specific algo's

In [ ]:
#| exporti
import hashlib
from pathlib import Path
import networkx as nx
from collections import defaultdict

In [ ]:
#| export
def md5(input):
    return hashlib.md5(input.encode('utf-8')).hexdigest()

In [ ]:
md5('bla')

'128ecf542a35ac5270a87dc740918404'

In [ ]:
#| export

def binarysearch(minim,maxim,function, flips_to_true=True, verbose=True): 
    """
     function needs to return a boolean whether the solution is ok
     this implementation is for function that starts with false for minim and flip to true
     for TTTTFFFF, pass set flips_to_true flag to false. This flag is important to set correct!
    """
    new = minim
    while True:
        new = (minim+maxim)//2
        if verbose: print(f'to_test: {new}, min {minim}, max {maxim} ', end=' ')
        res = function(new)
        if verbose: print('function returns', res)
        if not flips_to_true: res = not res
        if res:
            if new == maxim: # solution found
                if flips_to_true:
                    print('solution found',new)
                    return new
                else:
                    print('solution found',new-1)
                    return new-1
            maxim = new
        else: minim = new+1


In [ ]:
assert binarysearch(0,200, lambda x: x > 50, verbose=False) == 51
assert binarysearch(0,200, lambda x: x < 50, flips_to_true=False, verbose=False) == 49
assert binarysearch(0,201, lambda x: x > 50, verbose=False) == 51
assert binarysearch(0,201, lambda x: x < 50, flips_to_true=False, verbose=False) == 49
assert binarysearch(0,200, lambda x: x >= 50, verbose=False) == 50
assert binarysearch(0,200, lambda x: x <= 50, flips_to_true=False, verbose=False) == 50
assert binarysearch(0,201, lambda x: x >= 50, verbose=False) == 50
assert binarysearch(0,201, lambda x: x <= 50, flips_to_true=False, verbose=False) == 50

solution found 51
solution found 49
solution found 51
solution found 49
solution found 50
solution found 50
solution found 50
solution found 50


In [ ]:
#| export
def deduce_matches(input_dict, option_type=str):
    """
    Takes a dict with multiple keys that have one or more options
    The trick is to start with what you know: keys with one option and remove that option for the other keys
    Continuing that process leads to every key ending up with one option (hopefully)

    Assumes: the options are strings and stored in an interable
    """
    found = 0
    while found < len(input_dict):
        for k, v in input_dict.items():
            if not isinstance(v, option_type) and len(v) == 1: # found one
                to_rem = v.pop()
                input_dict[k] = to_rem
                found += 1
                for _ , v2 in input_dict.items(): # remove the item from other lists
                    if not isinstance(v2, option_type) and to_rem in v2:
                        v2.remove(to_rem)
    return input_dict

In [ ]:
meals = {'morning': ['yoghurt', 'lasagna', 'pizza'],
        'lunch': ['sandwich', 'lasagna'],
        'evening': ['pizza', 'lasagna'],
        'night': ['pizza']}
deduce_matches(meals)

{'morning': 'yoghurt',
 'lunch': 'sandwich',
 'evening': 'lasagna',
 'night': 'pizza'}

In [ ]:
#| export
def find_pattern_in_iter(start_pattern, function, goal = None, n_iter=1000000000):
    """
        Returns when a SPECIFIED pattern has been found from a function
        If goal = None, then first time the start pattern shows up again is returned
        Returns steps, pattern
    """
    if not goal: goal = start_pattern
    current = start_pattern
    for i in range(1,n_iter):
        current = function(current)
        # print(current)
        if current == goal:
            print(f'At step {i}, goal: {current} was found')
            return i, current

In [ ]:
#| export
def find_repeat(start_pattern, function, n_iter=None):
    """
        Returns when a NONSPECFIED repeating pattern has been found
        Returns steps, pattern
    """
    if not n_iter: n_iter = round(10e20)
    seen = {start_pattern}
    current = start_pattern
    for i in range(1,n_iter):
        current = function(current)
        # print(current)
        if current in seen:
            print(f'Repeat was found at step {i}. Pattern: {current}')
            return i,current
        seen.add(current)

In [ ]:
#| export
def find_cycle(start_pattern, function):
    """
        Find cycle length of some repeating pattern, by first inspecting which item repeats when
        And subtracting the time the item was first seen
    """
    step_second, pattern = find_repeat(start_pattern, function)
    step_first, pattern = find_pattern_in_iter(start_pattern, function, goal = pattern)
    return step_second - step_first

In [ ]:
#| hide
class Test_gen():
    def __init__(self):
        self.results = iter([5,10,15,5,99,10])
    def __call__(self,*args):
        return next(self.results)



In [ ]:
assert find_pattern_in_iter(99,Test_gen(),n_iter=10) == (5,99)
assert find_pattern_in_iter(99,Test_gen(),goal=10, n_iter=10) == (2,10)
assert find_pattern_in_iter(99,Test_gen(),goal=5, n_iter=10) == (1,5)
assert find_repeat(99,Test_gen(),n_iter=10) == (4,5)

found at step 5 99
found at step 2 10
found at step 1 5
found at step 4 5


In [ ]:
class UnionFind():
    # should have unique objects
    def __init__(self, it):
        self.parents = {obj:obj for obj in it}
        self.sizes = {obj:1 for obj in it}
        assert len(it) == len(self.parents), 'does your iterable contain duplicates?'
        
    def add(self, obj):
        # add a new object after instantiation, returns False if object already in
        if obj not in self.parents:
            self.parents[obj] = obj
            self.sizes[obj] = 1
            return True
        return False
        
    def _get_parent(self, x):
        # finding the parent of an object
        while x != self.parents[x]:
            parent = self.parents[x]
            # path compression
            self.parents[x] = self.parents[parent]
            x = parent
        return x
        
    def union(self,x,y):
        # unions two objects, returns False if items have the same parent and are therefore already in the same group
        for i in (x,y):
            if i not in self.parents:
                self.add(i)
                
        x,y  = self._get_parent(x), self._get_parent(y)
        if x == y:
            return False
        if self.sizes[x] < self.sizes[y]:
            # make sure that x is the largest group
            x, y = y, x
        self.parents[y] = x
        self.sizes[x] += self.sizes[y]
        self.sizes[y] = 0
    
    def groups(self):
        # returns all linked objects in a list of lists
        groups = defaultdict(list)
        for i in self.parents:
            groups[self._get_parent(i)].append(i)
        return list(groups.values())

In [ ]:
# tested above class on some leetcode problems: works!
uf = UnionFind([100,101,102,103])
uf.union(101,100)
uf.groups()


[[100, 101], [102], [103]]